# Convert COVID-19 data from CSSE to Gapminder format

load csv  data soruces: https://github.com/CSSEGISandData/COVID-19
CC-BY: Masaki Yamabe (masakiyamabe.com)

In [1]:
#Library
import numpy as np
import pandas as pd

source_confirmed = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
source_death = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv"
source_recoverd = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv"

df_confirmed = pd.read_csv(source_confirmed)
df_death = pd.read_csv(source_death)
df_recoverd  = pd.read_csv(source_recoverd)

In [2]:
#delete unuse column
del df_confirmed['Province/State']
del df_confirmed['Lat']
del df_confirmed['Long']

del df_death['Province/State']
del df_death['Lat']
del df_death['Long']

del df_recoverd['Province/State']
del df_recoverd['Lat']
del df_recoverd['Long']
df_recoverd.head()

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20
0,Thailand,0,0,0,0,2,2,5,5,5,...,31,31,33,34,34,35,35,35,35,41
1,Japan,0,0,0,0,1,1,1,1,1,...,76,76,101,118,118,118,118,118,144,144
2,Singapore,0,0,0,0,0,0,0,0,0,...,78,78,78,96,96,97,105,105,109,114
3,Nepal,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,Malaysia,0,0,0,0,0,0,0,0,0,...,24,24,24,26,26,26,35,42,42,49


In [3]:
# group by "country/region"
df_confirmed = df_confirmed.groupby("Country/Region").apply(lambda x: x.sum()).drop('Country/Region', axis=1).reset_index()
df_death =  df_death.groupby("Country/Region").apply(lambda x: x.sum()).drop('Country/Region', axis=1).reset_index()
df_recoverd =  df_recoverd.groupby("Country/Region").apply(lambda x: x.sum()).drop('Country/Region', axis=1).reset_index()

In [4]:
#insert case column

df_confirmed.insert(1,"case",'confirmed')
df_death.insert(1,"case",'deaths')
df_recoverd.insert(1,"case",'recoverd')

In [5]:
df_combined = pd.concat([df_confirmed, df_death, df_recoverd])
df_combined.head()

,Country/Region,case,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20
0,Afghanistan,confirmed,0,0,0,0,0,0,0,0,...,4,4,5,7,7,7,11,16,21,22
1,Albania,confirmed,0,0,0,0,0,0,0,0,...,0,2,10,12,23,33,38,42,51,55
2,Algeria,confirmed,0,0,0,0,0,0,0,0,...,19,20,20,20,24,26,37,48,54,60
3,Andorra,confirmed,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,2,39
4,Antigua and Barbuda,confirmed,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1


In [6]:
#convert date format
from datetime import datetime as dt
cols = np.array(df_combined.columns)


for i,word in enumerate(cols):
    if i>1:
        s = cols[i]
        tdatetime = dt.strptime(s, '%m/%d/%y')
        cols[i] = tdatetime.strftime('%Y%m%d')
        
# set new columns to df
df_combined.columns = cols
df_combined.head()

,Country/Region,case,20200122,20200123,20200124,20200125,20200126,20200127,20200128,20200129,...,20200308,20200309,20200310,20200311,20200312,20200313,20200314,20200315,20200316,20200317
0,Afghanistan,confirmed,0,0,0,0,0,0,0,0,...,4,4,5,7,7,7,11,16,21,22
1,Albania,confirmed,0,0,0,0,0,0,0,0,...,0,2,10,12,23,33,38,42,51,55
2,Algeria,confirmed,0,0,0,0,0,0,0,0,...,19,20,20,20,24,26,37,48,54,60
3,Andorra,confirmed,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,2,39
4,Antigua and Barbuda,confirmed,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1


In [7]:
df_combined.describe()

,20200122,20200123,20200124,20200125,20200126,20200127,20200128,20200129,20200130,20200131,...,20200308,20200309,20200310,20200311,20200312,20200313,20200314,20200315,20200316,20200317
count,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,...,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000,456.000000
mean,1.315789,1.537281,2.199561,3.322368,4.881579,6.732456,12.754386,14.089912,18.745614,22.723684,...,382.217105,394.831140,410.653509,433.076754,441.638158,484.315789,514.342105,548.076754,584.967105,626.953947
std,25.703231,30.150296,43.125904,65.886843,97.219516,134.791523,258.068789,285.140063,381.333388,459.182769,...,4671.441320,4720.435238,4767.374188,4826.384583,4867.117631,4952.609674,5037.890988,5129.598975,5216.159370,5315.703096
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.250000,3.000000,4.000000,6.000000,6.250000,8.250000,12.000000,14.000000,17.000000,26.000000
max,548.000000,643.000000,920.000000,1406.000000,2075.000000,2877.000000,5509.000000,6087.000000,8141.000000,9802.000000,...,80823.000000,80860.000000,80887.000000,80921.000000,80932.000000,80945.000000,80977.000000,81003.000000,81033.000000,81058.000000


In [8]:
df_combined.to_csv("covid19_gapminder.csv",index=False)